In [1]:
def solve(filename):
    map =[]
    dirs =""
    with open(filename) as f:
        tmp  = f.read().split("\n\n")
        map =[list(line)for line in tmp[0].splitlines()]
        dirs = "".join(tmp[1].split("\n"))
        del tmp
    
    d_key = {">": (0,1), "<": (0,-1), "^": (-1, 0), "v": (1,0)}
    p = [[i, line.index("@")] for i, line in enumerate(map) if line.count("@") > 0][0]
    get_nxt = lambda p, d: [i + j for i,j in zip(p, d)]
    v = lambda p: map[p[0]][p[1]]
    def set_v(p, nxt_p):
        map[nxt_p[0]][nxt_p[1]] = "@"
        map[p[0]][p[1]] = "."
    def mk_move(dir):
        d = d_key[dir]
        nxt_p = get_nxt(p, d)
        nxt_v  = v(nxt_p)
        if nxt_v == "O":
            inFrnt = []
            if d[0] == 0:
                inFrnt = map[p[0]][p[1]+1:-1] if d[1] > 0 else map[p[0]][p[1]-1:0:-1]
            else:
                tmp = [line[p[1]] for line in map]   
                inFrnt = tmp[p[0]+1:-1] if d[0] > 0 else  tmp[p[0]-1:0:-1]
                del tmp
            if inFrnt.count(".") > 0:
                delta = inFrnt.index(".") + 1
                if inFrnt[:delta].count("#") == 0:
                    map[p[0]+delta*d[0]][p[1]+delta*d[1]] = "O"
                    set_v(p, nxt_p)
                    return nxt_p            
        if nxt_v == ".":
            set_v(p, nxt_p)
            return nxt_p
        else:
            return p
    for dir in dirs:
        p =  mk_move(dir)

    out = sum([i*100 + j for i, line in enumerate(map) for j, v in enumerate(line) if v == "O"])
    return out
print(f"part 1  - test: {solve("./test15.txt")}")
print(f"part 1  - test 2: {solve("./test15-2.txt")}")
print(f"part 1 - result: {solve("./day15.txt")}")

part 1  - test: 2028
part 1  - test 2: 10092
part 1 - result: 1406392


In [78]:

def solve2(filename):
    map = []
    dirs = ""
    map_dict = {"O": "[]", "@": "@.", ".": "..", "#": "##"}
    tmp = open(filename).read().split("\n\n")
    map = [list("".join( [map_dict[c] for c in line])) for line in tmp[0].splitlines()]
    dirs = tmp[1].replace("\n","")
    del tmp
    d_key = {">": (0,1), "<": (0,-1), "^": (-1, 0), "v": (1,0)}
    p = [[i, line.index("@")] for i, line in enumerate(map) if line.count("@") > 0][0]
    get_nxt = lambda p, d: [i + j for i,j in zip(p, d)]
    v = lambda p: map[p[0]][p[1]]
    def set_v(p, nxt_p):
        map[nxt_p[0]][nxt_p[1]] = "@"
        map[p[0]][p[1]] = "."
    def get_boxes_vert(lhp,d):
        nxt_ps = [get_nxt([lhp[0],lhp[1]+x],d) for x in [-1,0,1]]
        if all([v(p) == "." for p in nxt_ps[1:]]):
            return [lhp]
        boxes = []
        if all([v(p) != "#" for p in nxt_ps[1:]]):
            for p in nxt_ps:
                if v(p) == "[":
                    tmp = get_boxes_vert(p,d)
                    if tmp == []:
                        return []
                    else:
                        boxes.extend(tmp)
            boxes.extend([lhp])
        return boxes
    def get_boxes_hor(lhp,d):
        nxt_p = [lhp[0],lhp[1] +{1: 2, -1: -1}[d[1]]]
        if v(nxt_p) == ".":
            return [lhp]
        boxes = []
        if v(nxt_p) != "#":
            if v(nxt_p) == "[":
                 boxes.extend(get_boxes_hor(nxt_p,d))
            else:
                boxes.extend(get_boxes_hor([nxt_p[0],nxt_p[1]-1],d))
            if boxes == []:
                return []
            boxes.extend([lhp])
        return boxes
    def move_boxes(boxes, d):
        idx = 1 if d[0] == 0 else 0 
        ordered = sorted(boxes.copy(), key=lambda lhp: lhp[idx], reverse=d[idx] > 0)
        for lhp in ordered:
            ps = [[lhp[0],lhp[1]+x] for x in [0,1]]
            vs = [v(p) for p in ps]
            nxt_ps = [get_nxt(p, d) for p in ps]
            for p in ps:
                map[p[0]][p[1]] = '.'
            for x in [0,1]:
                map[nxt_ps[x][0]][nxt_ps[x][1]] = vs[x]
    def mk_move(dir):
        d = d_key[dir]
        nxt_p = get_nxt(p, d)
        nxt_v  = v(nxt_p)
        if nxt_v == "[" or nxt_v == "]":
            inFrnt = []
            if d[0] == 0:
                boxes = get_boxes_hor(nxt_p if nxt_v == "[" else [nxt_p[0], nxt_p[1] - 1], d)
            else:
                boxes = get_boxes_vert(nxt_p if nxt_v == "[" else [nxt_p[0], nxt_p[1] - 1], d)
            if boxes == []:
                return p
            else:
                boxes = list({tuple(b) for b in boxes})
                move_boxes(boxes, d)
                set_v(p, nxt_p)
                return nxt_p  
        if nxt_v == ".":
            set_v(p, nxt_p)
            return nxt_p
        else:
            return p
    for dir in dirs:
        p =  mk_move(dir)
    out = sum([i*100 + j if v == "[" else 0 for i, line in enumerate(map) for j, v in enumerate(line) ])
    # for line in map: print("".join(line))
    return out
# print(f"part 2  - test 2: {solve2("./test15-2.txt")}")
# print(f"part 2  - test 3: {solve2("./test15-3.txt")}")
print(f"part 2 - result: {solve2("./day15.txt")}")
# 1417658 is too low

part 2 - result: 1429013
